In [112]:
import os
import pandas as pd
import numpy as np

In [113]:
df = pd.read_csv("product_sales.csv")
df.head()

,item_id,unit_price,unit_cost,units_sold,incr_cvr,incr_sales
0,1,95,92,0.6,0.323,0.08906
1,2,37,27,0.8,0.106,0.01991
2,3,34,26,0.6,0.328,0.03778
3,4,32,30,0.7,0.000,0.08562
4,5,62,45,1.0,0.079,0.02637


In [143]:
perc =[.20, .40, .60, .80]
include =['object', 'float', 'int']
desc = df.describe(percentiles = perc, include = include)
desc

,item_id,unit_price,unit_cost,units_sold,incr_cvr,incr_sales
count,250.000000,250.000000,250.000000,250.000000,250.000000,250.000000
mean,125.500000,60.620000,51.596000,1.500400,0.136608,0.050986
std,72.312977,28.318959,24.182223,0.839511,0.157928,0.029084
min,1.000000,10.000000,9.000000,0.100000,0.000000,0.000370
20%,50.800000,33.800000,28.000000,0.600000,0.000000,0.020262
40%,100.600000,49.000000,43.000000,1.200000,0.005000,0.043162
50%,125.500000,57.000000,50.500000,1.400000,0.069000,0.051620
60%,150.400000,69.000000,57.400000,1.800000,0.135000,0.061298
80%,200.200000,89.000000,75.000000,2.300000,0.288400,0.080604
max,250.000000,110.000000,105.000000,3.100000,0.615000,0.099430


In [114]:
def calc_profit(increment):
    original_unit_price = df['unit_price']
    #increment => {-0.1,0.2}
    price_change_multiplier = 1 + increment
    #For iterantion#1 increment = 0, price_change_multiplier = 1
    price =  original_unit_price * price_change_multiplier
    
    original_units_sold=df['units_sold']
    additional_units = df['incr_sales'] * increment * 10
    #For iteration#1 additional_units = 0
    volumes = original_units_sold * (1-additional_units)
    
    original_cost = df['unit_cost']
    initial_incr_cvr = 0.05
    
    initial_profit= (np.sum(price*volumes)-np.sum(original_cost*volumes))
    profit_multiplier = np.prod(1-(df['incr_cvr'] * increment * 10))
    #For iterantion#1 increment =0, profit_multiplier = 1

    net_profit = np.sum(initial_profit * profit_multiplier)
    return net_profit

In [138]:
def incremental_new(initial_increments):
    initial_rev = calc_profit(initial_increments)
   # print('initial incr =',initial_increments)
    print('initial rev =',initial_rev)
    intermediate_rev = 0
    rev = 0
    
    for i in range(0,250):
       # print(‘CURRENT I’, i, increments)
        increments = initial_increments
        print('increments[i]',increments[i],'initial_increments[i]',initial_increments[i])
        
        if increments[i] > -0.095:
            boost = 0
            if incr_cvr_array[i] > 0.136608 and incr_sales_array[i] > 0.05098572:
                boost = 0.04 #keep price low
            elif incr_cvr_array[i] > 0.136608 and incr_sales_array[i] < 0.05098572:
                boost = 0.015
            elif incr_cvr_array[i] < 0.136608 and incr_sales_array[i] > 0.05098572:
                boost = 0.02
            else:
                boost = 0.05  
            increments[i] = increments[i]-boost
            rev = calc_profit(increments)
            print(i,"{:.3f}".format(increments[i]),rev)
        
        if rev > initial_rev:
            final_increments = increments
            intermediate_rev = rev
            
        if increments[i] < 0.15:
            boost = 0
            if incr_cvr_array[i] > 0.136608 and incr_sales_array[i] > 0.05098572:
                boost = 0.01 #keep price low
            elif incr_cvr_array[i] > 0.136608 and incr_sales_array[i] < 0.05098572:
                boost = 0.015
            elif incr_cvr_array[i] < 0.136608 and incr_sales_array[i] > 0.05098572:
                boost = 0.02
            else:
                boost = 0.05
            increments[i] = initial_increments[i] + boost
            rev = calc_profit(increments)   
            print(i,"{:.3f}".format(increments[i]),rev)
            
        if (rev > max(initial_rev,intermediate_rev)):
            final_increments = increments
            intermediate_rev = rev
            
    #print('final incr =',final_increments)
    print('final rev = ',intermediate_rev)
    return final_increments

In [146]:
incr_cvr_array = df['incr_cvr'].to_numpy()
np.mean(incr_cvr_array)

0.136608

In [145]:
incr_sales_array = df['incr_sales'].to_numpy()
np.mean(incr_sales_array)

0.05098572

In [139]:
flag = 0
increment = np.zeros(df.shape[0], dtype=np.float64)
increment_i = increment
#flag = 1 is a condition when the increment vector remains the same


while flag < 2:
   # print(‘Initial overall profit:’,find_rev(increment_i))
    increment_iplus1 = incremental_new(increment_i,incr_cvr_array,incr_sales_array)
    #if (min(increment_iplus1) == increment_i):
        #if(min(increment_iplus1)==1):
    flag = flag +1
    print(flag)
    increment_i = increment_iplus1

print('Product changes', increment_i)
print('New net profit',calc_profit(increment_i)) 
#print('Old net profit',calc_profit(increment)) 
price = increment_i
#write.csv(price,"price.csv")

initial rev = 3285.899999999998
increments[i] 0.0 initial_increments[i] 0.0
0 -0.050 3813.2082673252535
0 0.000 3285.899999999998
increments[i] 0.0 initial_increments[i] 0.0
1 -0.050 3458.5626066497985
1 0.000 3285.899999999998
increments[i] 0.0 initial_increments[i] 0.0
2 -0.050 3823.6834344887966
2 0.000 3285.899999999998
increments[i] 0.0 initial_increments[i] 0.0
3 -0.050 3284.7919868000026
3 0.000 3285.899999999998
increments[i] 0.0 initial_increments[i] 0.0
4 -0.050 3412.6611107242516
4 0.000 3285.899999999998
increments[i] 0.0 initial_increments[i] 0.0
5 -0.050 3282.060952349999
5 0.000 3285.899999999998
increments[i] 0.0 initial_increments[i] 0.0
6 -0.050 3277.1195228750003
6 0.000 3285.899999999998
increments[i] 0.0 initial_increments[i] 0.0
7 -0.050 3745.340766316803
7 0.000 3285.899999999998
increments[i] 0.0 initial_increments[i] 0.0
8 -0.050 4073.790641195702
8 0.000 3285.899999999998
increments[i] 0.0 initial_increments[i] 0.0
9 -0.050 3280.6639160000013
9 0.000 3285.8999